# 🧑‍🍳 Cooking up flavorful SWOTs with the kernel

## 🔥 Get a kernel ready

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from IPython.display import display, Markdown

kernel = sk.Kernel()

useAzureOpenAI = False

if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("azureopenai", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("openai", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))

print("A kernel is now ready.")    

## 🧂 Create an "inline" semantic function

Semantic Kernel uses a simple templating format where variables are denoted with `{{` and `}}` and a dollar sign before the name. The default variable name is `$input`.

In [ ]:
sk_prompt = """
{{$input}}

Summarize the content above in less than 140 characters.
"""

summary_function = kernel.create_semantic_function(prompt_template=sk_prompt,
                                                    description="Summarizes the input to length of an old tweet.",
                                                    max_tokens=200,
                                                    temperature=0.1,
                                                    top_p=0.5)

print("A semantic function for summarization has been registered.");

## 🏎️ Set the default variable $input and have the kernel run it

Note that `$input` is the default variable name and it gets fed into the function from the `input_str` parameter.

In [ ]:
sk_input = """
Let me illustrate an example. Many weekends, I drive a few minutes from my house to a local pizza store to buy 
a slice of Hawaiian pizza from the gentleman that owns this pizza store. And his pizza is great, but he always 
has a lot of cold pizzas sitting around, and every weekend some different flavor of pizza is out of stock. 
But when I watch him operate his store, I get excited, because by selling pizza, he is generating data. 
And this is data that he can take advantage of if he had access to AI.

AI systems are good at spotting patterns when given access to the right data, and perhaps an AI system could spot 
if Mediterranean pizzas sell really well on a Friday night, maybe it could suggest to him to make more of it on a 
Friday afternoon. Now you might say to me, "Hey, Andrew, this is a small pizza store. What's the big deal?" And I 
say, to the gentleman that owns this pizza store, something that could help him improve his revenues by a few 
thousand dollars a year, that will be a huge deal to him.
""";
# Text source: https://www.ted.com/talks/andrew_ng_how_ai_could_empower_any_business/transcript

summary_result = await kernel.run_async(summary_function, input_str=sk_input) 

display(Markdown("### ✨ " + str(summary_result)))

There's a simpler way to run the function that involves just calling it directly:

In [ ]:
summary_result = summary_function(sk_input)

display(Markdown("### ✨ " + str(summary_result)))

## 🧂 Create an 'inline' native function

It's also possible to create functions that the kernel can use that do not use LLM prompts and can instead use good ole fashion code. Native code isn't as fancy as semantic code — so the best we will try to do here is turn an input text into ... pig latin.

In [ ]:
from semantic_kernel.skill_definition import (
    sk_function,
    sk_function_context_parameter,
)
from semantic_kernel.orchestration.sk_context import SKContext

class ExoticLanguagePlugin:
    def word_to_pig_latin(self, word):
        vowels = "AEIOUaeiou"
        if word[0] in vowels:
            return word + "way"
        for idx, letter in enumerate(word):
            if letter in vowels:
                break
        else:
            return word + "ay"
        return word[idx:] + word[:idx] + "ay"
    @sk_function(
        description="Takes text and converts it to pig latin",
        name="pig_latin",
        input_description="The text to convert to pig latin",
    )
    def pig_latin(self, sentence:str) -> str:
        words = sentence.split()
        pig_latin_words = []
        for word in words:
            pig_latin_words.append(self.word_to_pig_latin(word))
        return ' '.join(pig_latin_words)

exotic_language_plugin = kernel.import_skill(ExoticLanguagePlugin(), skill_name="exotic_language_plugin")
pig_latin_function = exotic_language_plugin["pig_latin"]


## 🏎️ Run the semantic function's output through the native function's input

To chain two functions together we simply add the function appropriately. And we unfortunately butcher the output.

In [ ]:
final_result = await kernel.run_async(summary_function, pig_latin_function, input_str=sk_input) 

display(Markdown("### ✨ " + str(final_result)))

## 📈 Let's get down to business

[![](assets/andrew_ng.jpg)](assets/thepizzastore720.mp4)

In his TED 2022 talk, Andrew Ng describes "How AI could empower any business ([TED 2022](https://www.ted.com/talks/andrew_ng_how_ai_could_empower_any_business))" as  means to help us all understand that the new kind of AI technology could be put into service for anybody — including Andrew's favorite pizza shop.

There are a variety of business strategy questions that can be asked with LLMs, and so we'll use Semantic Kernel to work our way towards repeatable processes that can be used for other businesses in the future. We want to get a sense of the challenges of the pizza shop owner, so we take the time to interview them to best understand their Strengths, Weaknesses, Opportunities, and Threats -- so-called "SWOT Analysis." 

By having a few interview questions to get going, there's the potential to understand the pizza shop owner's business better.

1. **Strengths**
    - What unique recipes or ingredients does the pizza shop use?
    - What are the skills and experience of the staff?
    - Does the pizza shop have a strong reputation in the local area?
    - Are there any unique features of the shop or its location that attract customers?
2. **Weaknesses**
    - What are the operational challenges of the pizza shop? (e.g., slow service, high staff turnover)
    - Are there financial constraints that limit growth or improvements?
    - Are there any gaps in the product offering?
    - Are there customer complaints or negative reviews that need to be addressed?
3. **Opportunities**
    - Is there potential for new products or services (e.g., catering, delivery)?
    - Are there under-served customer segments or market areas?
    - Can new technologies or systems enhance the business operations?
    - Are there partnerships or local events that can be leveraged for marketing?
4. **Threats**
    - Who are the major competitors and what are they offering?
    - Are there potential negative impacts due to changes in the local area (e.g., construction, closure of nearby businesses)?
    - Are there economic or industry trends that could impact the business negatively (e.g., increased ingredient costs)?
    - Is there any risk due to changes in regulations or legislation (e.g., health and safety, employment)?
 
The answers that came back are organized in a so-called "SWOT" matrix.

### 🔖 Strengths and Weaknesses

There's a lot for the pizza shop owner to build upon (their strengths) and there are some ongoing concerns (their weaknesses) that may slow the pizza shop owner's business over time. 

| **Strengths**                                     | **Weaknesses**                                               |
| --- | --- |
| Unique garlic pizza recipe that wins top awards  | High staff turnover                                          |
| Owner trained in Sicily at some of the best pizzerias                          | Floods in the area damaged the seating areas that are in need of repair  |
| Strong local reputation                           | Absence of popular calzones from menu                        |
| Prime location on university campus               | Negative reviews from younger demographic for lack of hip ingredients |

Meanwhile there's money being left on the table (their opportunities) with calamities (their threats) waiting in the wings to possibly happen and may knock their pizza shop out cold.

### 🔖 Opportunities and Threats

| **Opportunities**                                 | **Threats**                                                  |
| --- | ---|
| Untapped catering potential                       | Rising competition from cheaper pizza businesses nearby |
| Growing local tech startup community              | There's nearby street construction that will impact foot traffic |
| Unexplored online presence and order capabilities | Rising cost of cheese                                        |
| Upcoming annual food fair                         | No immediate local regulatory changes but it's election season |

## 🤷 What does this have to do with LLMs and Semantic Kernel?

It's no secret that one prompt isn't as good as chaining multiple prompts together. Let's do that in Semantic Kernel to illustrate this commonly used technical concept while also demonstrating how a business strategist might play with the building blocks of this new kind of AI by: 1/ reusing past work, and 2/ adapting it to a different audience. 

Let's begin with a simple prompt, and then show you how to add another. And note that we're expanding the number of context variables to use in the template with this example.

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from IPython.display import display, Markdown

kernel = sk.Kernel()

useAzureOpenAI = False

if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("azureopenai", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("openai", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))

swot_interview= """
1. **Strengths**
    - What unique recipes or ingredients does the pizza shop use?
    - What are the skills and experience of the staff?
    - Does the pizza shop have a strong reputation in the local area?
    - Are there any unique features of the shop or its location that attract customers?
2. **Weaknesses**
    - What are the operational challenges of the pizza shop? (e.g., slow service, high staff turnover)
    - Are there financial constraints that limit growth or improvements?
    - Are there any gaps in the product offering?
    - Are there customer complaints or negative reviews that need to be addressed?
3. **Opportunities**
    - Is there potential for new products or services (e.g., catering, delivery)?
    - Are there under-served customer segments or market areas?
    - Can new technologies or systems enhance the business operations?
    - Are there partnerships or local events that can be leveraged for marketing?
4. **Threats**
    - Who are the major competitors and what are they offering?
    - Are there potential negative impacts due to changes in the local area (e.g., construction, closure of nearby businesses)?
    - Are there economic or industry trends that could impact the business negatively (e.g., increased ingredient costs)?
    - Is there any risk due to changes in regulations or legislation (e.g., health and safety, employment)?"""


sk_prompt = """
{{$input}}

Convert the analysis provided above to the business domain of {{$domain}}.
"""
shift_domain_function = kernel.create_semantic_function(prompt_template=sk_prompt,
                                                    description="Translate an idea to another domain.",
                                                    max_tokens=1000,
                                                    temperature=0.1,
                                                    top_p=0.5)
my_context = kernel.create_new_context()

my_context['input'] = swot_interview
my_context['domain'] = "construction management"

result = await kernel.run_async(shift_domain_function, input_context=my_context)

display(Markdown(f"### ✨ Shift the SWOT interview questions to the world of {my_context['domain']}\n"+ str(result)))


With another semantic function that will convert whatever it receives into the reading level of `level` the first result can get chained into the second result.

In [ ]:
sk_prompt = """
{{$input}}

Rewrite the text above to be understood by a {{$level}}.
"""
shift_reading_level_function = kernel.create_semantic_function(prompt_template=sk_prompt,
                                                    description="Change the reading level of a given text.",
                                                    max_tokens=1000,
                                                    temperature=0.1,
                                                    top_p=0.5)

my_context['input'] = swot_interview
my_context['domain'] = "construction management"
my_context["level"] = "child"

result = await kernel.run_async(shift_domain_function, shift_reading_level_function, input_context=my_context)

display(Markdown(f"### ✨ Shift the SWOT interview questions to the world of {my_context['domain']} at the level of {my_context['level']}\n"+ str(result)))

I think that covers enough to get us going for now. Are you ready?

## 🍕 Let's get our mind back in the pizza shop owner's mindset

We can choose to build on a few of the pizza shop owner's existing strengths to grow their business. This is generally the most prudent course of action because it's what's typically considered "a good offensive maneuver" — because after all, you're more likely to win where you're winning the most. 

| SAMPLE STRENGTH 1 | SAMPLE STRENGTH 2 |
|---|---|
| The best garlic pizza in the business | The store owner was trained in Sicily |
| ![](assets/garlicpizza.png) | ![](assets/measuring.png) |

We can also help make the business more efficient by addressing a few of their weaknesses. This is more of a defensive approach which plays well when a business has a working offense. Starting with one's strengths and then addressing a weakness tends to be a better tactic — but then again, sometimes a business' weakness can kill it so quickly that its strengths never get dialed up in time. Keep in mind that the circumstances are always fluid, and so being *agile* will always matter.

| SAMPLE WEAKNESS 1 | SAMPLE WEAKNESS 2 |
|---|---|
| High turnover in under-skilled employees | Calzones are trending but not on the menu | 
| ![](assets/workersleaving.png) | ![](assets/calzones.png) |

Lastly, with the right balance of leveraging strengths and addressing weaknesses, there may be room for the business to investigate opportunities to grow the business by reaching into new markets with product modifications and/or by launching entirely new products. And as the business attempts to take on more uncertainty and risk, it's prudent for the business to imagine and prepare for how to make itself more resilient. By being more resilient, it becomes possible to take on more risk to reap more rewards. In absence of designing itself for resilience, the business can take on too much risk and fold when an unexpected failure occurs.

| SAMPLE OPPORTUNITY 1 | SAMPLE THREAT 1 |
|---|---|
| The spring neighborhood fair is coming | Cheese prices are skyrocketing in price |
| ![](assets/townfair.png) | ![](assets/cheese.png) |

Are you getting a good feel for the business of the pizza owner? And are you also feeling confident that you could easily shift into a different business vertical with the unusual cognitive elasticity afforded to you by LLMs? If not, change the interview responses or the industry vertical.

## 🔖 Reminder: All ✨ generative responses result from having the model fill in the _____.

![](assets/completion.png)